<a href="https://colab.research.google.com/github/linlcc/NLP/blob/master/3_Viterbi_Decoder_and_Evaluation_for_Sequence_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, I will build a Viterbi decoder for an LSTM named-entity recognition model. As we knew, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. I cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

Therefore, I will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, I will also implement some functions to evaluate the output of an NER system according to two metrics:
1. I will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. I will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [0]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=71f213ebba9ef790503decf099282c99dc0872a53ff19a09dfe5316f8891b6b6
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
!pip install --upgrade spacy allennlp
import spacy
print(spacy.__version__)

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
Requirement already up-to-date: allennlp in /usr/local/lib/python3.6/dist-packages (0.9.0)
2.2.4


In [0]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003
import math
# from seqeval.metrics import precision_score, recall_score, f1_score, classification_report,accuracy_score
torch.manual_seed(1)

In [0]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [0]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

159377B [00:00, 2518823.43B/s]
562it [00:00, 4742.04it/s]
8366B [00:00, 7638125.22B/s]
23it [00:00, 5124.51it/s]
100%|██████████| 585/585 [00:00<00:00, 61961.31it/s]


## Define and Train Model

In [0]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9032 ||: 100%|██████████| 281/281 [00:01<00:00, 177.60it/s]
accuracy: 0.7878, loss: 1.1820 ||: 100%|██████████| 12/12 [00:00<00:00, 445.38it/s]
accuracy: 0.8442, loss: 0.7304 ||: 100%|██████████| 281/281 [00:01<00:00, 248.16it/s]
accuracy: 0.7878, loss: 1.1768 ||: 100%|██████████| 12/12 [00:00<00:00, 462.55it/s]
accuracy: 0.8442, loss: 0.7167 ||: 100%|██████████| 281/281 [00:01<00:00, 253.97it/s]
accuracy: 0.7878, loss: 1.1778 ||: 100%|██████████| 12/12 [00:00<00:00, 389.01it/s]
accuracy: 0.8442, loss: 0.7068 ||: 100%|██████████| 281/281 [00:01<00:00, 253.82it/s]
accuracy: 0.7878, loss: 1.1786 ||: 100%|██████████| 12/12 [00:00<00:00, 458.96it/s]
accuracy: 0.8442, loss: 0.6976 ||: 100%|██████████| 281/281 [00:01<00:00, 253.20it/s]
accuracy: 0.7878, loss: 1.1514 ||: 100%|██████████| 12/12 [00:00<00:00, 461.05it/s]
accuracy: 0.8442, loss: 0.6904 ||: 100%|██████████| 281/281 [00:01<00:00, 256.68it/s]
accuracy: 0.7878, loss: 1.1439 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 99,
 'best_validation_accuracy': 0.8775510204081632,
 'best_validation_loss': 0.37447693160599255,
 'epoch': 99,
 'peak_cpu_memory_MB': 2564.34,
 'peak_gpu_0_memory_MB': 723,
 'training_accuracy': 0.9270204427497087,
 'training_cpu_memory_MB': 2564.34,
 'training_duration': '0:01:59.965550',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 723,
 'training_loss': 0.18652244034290374,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8775510204081632,
 'validation_loss': 0.37447693160599255}

## Evaluation

The code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]

Now, implement two evaluation functions: `violations` and `span_stats`.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from collections import defaultdict
import numpy as np

def get_entities(seq, suffix=False):

    # for nested list
    if any(isinstance(s, list) for s in seq):
        seq = [item for sublist in seq for item in sublist + ['O']]

    prev_tag = 'O'
    prev_type = ''
    begin_offset = 0
    chunks = []
    Q=0
    for i, chunk in enumerate(seq + ['O']):
        if suffix:
            tag = chunk[-1]
            type_ = chunk.split('-')[0]
        else:
            tag = chunk[0]
            type_ = chunk.split('-')[-1]
        if end_of_chunk(prev_tag, tag, prev_type, type_,Q):
            chunks.append((prev_type, begin_offset, i-1))
        if start_of_chunk(prev_tag, tag, prev_type, type_,Q)[0]:      
            begin_offset = i
            Q=start_of_chunk(prev_tag, tag, prev_type, type_,Q)[1]
        prev_tag = tag
        prev_type = type_
    return chunks

def end_of_chunk(prev_tag, tag, prev_type, type_,Q):

    chunk_end = False
    if prev_tag == 'B' and tag == 'B': chunk_end = True
    if prev_tag == 'B' and tag == 'O': chunk_end = True
    if prev_tag == 'I' and tag == 'B': chunk_end = True
    if prev_tag != 'O' and prev_tag != '.' and prev_type != type_:
        chunk_end = True

    if prev_tag == 'I' and tag == 'I'and Q==1: chunk_end = False
    if prev_tag == 'I' and tag == 'O' and Q==1: chunk_end = False

    return chunk_end

def start_of_chunk(prev_tag, tag, prev_type, type_,Q):

    chunk_start = False
    if tag == 'B': 
      chunk_start = True
      Q=0
    if prev_tag == 'B' and tag == 'I' and prev_type!=type_: 
      chunk_start = True
      Q=1
    if prev_tag == 'I' and tag == 'I' and prev_type!=type_: 
      chunk_start = True
      Q=1
    if prev_tag == 'O' and tag == 'I' and prev_type!=type_: 
      chunk_start = True
      Q=1

    # print(Q)
    # if tag != 'O' and tag != '.' and prev_type != type_:
    #     chunk_start = True

    return chunk_start,Q

def f1_score(y_true, y_pred, average='micro', suffix=False):
   
    true_entities = set(get_entities(y_true, suffix))
    pred_entities = set(get_entities(y_pred, suffix))

    nb_correct = len(true_entities & pred_entities)
    nb_pred = len(pred_entities)
    nb_true = len(true_entities)

    p = nb_correct / nb_pred if nb_pred > 0 else 0
    r = nb_correct / nb_true if nb_true > 0 else 0
    score = 2 * p * r / (p + r) if p + r > 0 else 0

    return score

def precision_score(y_true, y_pred, average='micro', suffix=False):
   
    true_entities = set(get_entities(y_true, suffix))
    pred_entities = set(get_entities(y_pred, suffix))

    nb_correct = len(true_entities & pred_entities)
    nb_pred = len(pred_entities)

    score = nb_correct / nb_pred if nb_pred > 0 else 0

    return score

def recall_score(y_true, y_pred, average='micro', suffix=False):
  
    true_entities = set(get_entities(y_true, suffix))
    pred_entities = set(get_entities(y_pred, suffix))

    nb_correct = len(true_entities & pred_entities)
    nb_true = len(true_entities)

    score = nb_correct / nb_true if nb_true > 0 else 0

    return score


In [0]:
# count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input

def violations(tagged):
  i=0
  for item in tagged:
    for count in range(len(item)-1):
      if (item[count][2][0]=='O') and (item[count+1][2][0]=='I'):
        i=i+1
      elif (item[count][2][0]=='B') and (item[count+1][2][0]=='I') and (item[count][2][2:]!=item[count+1][2][2:]):
        i=i+1
  return i

# return the span-level precision, recall, and F1
# Take tagger output as input
def span_stats(tagged):
  y_true=[]
  y_pred=[]
  for item in tagged:
    y1=[]
    y2=[]
    for count in range(len(item)):
      y1.append(item[count][1])
      y2.append(item[count][2])
    y_true.append(y1)
    y_pred.append(y2)
  precision=precision_score(y_true, y_pred)
  recall=recall_score(y_true, y_pred)
  f1=f1_score(y_true, y_pred)
  # acc=accuracy_score(y_true, y_pred)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}
## check how many violations are made by the model output in predictor.
# print(violations(baseline_output))
print("violations: %d"  % violations(baseline_output))
print(span_stats(baseline_output))

violations: 30
{'precision': 0.2727272727272727, 'recall': 0.20930232558139536, 'f1': 0.23684210526315788}


## Decoding

Finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, construct a transition matrix. I use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [0]:
# This code show how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))

{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [0]:
# Creating transition matrix
tags=vocab.get_index_to_token_vocabulary('labels')

def replace_tags(search_value):
  for key, value in tags.items():
    if value==search_value:
      return key

tags2=[*tags.values()]

M = [[1]*len(tags) for _ in range(len(tags))]

for i in tags2:
  for j in tags2:
    if (i[0]=='O') and (j[0]=='I'):
      M[replace_tags(i)][replace_tags(j)]=float("-inf")
    elif (i[0]=='B') and (j[0]=='I') and (i[2:]!=j[2:]):
      M[replace_tags(i)][replace_tags(j)]=float("-inf")
    elif (i[0]=='I') and (j[0]=='I') and (i[2:]!=j[2:]):
      M[replace_tags(i)][replace_tags(j)]=float("-inf")
    else:
      M[replace_tags(i)][replace_tags(j)]=0

# for row in M:
#     print(row)

In [0]:
# adding emission probabilities with transition matrix
def test(s):
  item=predictor.predict_instance(s)['tag_logits']
  x=[]
  x1=[]
  for j in item:
    y=[]
    if item.index(j)==0: ## first word of every sentences -- do nothing
      for k in j:
        y.append(k+0)
      find_ind=np.argmax(y,axis=-1)
    else:
      for k in j:
        modify=k+M[find_ind][j.index(k)]
        y.append(modify)     
      find_ind=np.argmax(y,axis=-1)  
    x.append(y)
    x1.append(tags.get(find_ind))
    
  new = zip(s['tokens'], s['tags'], [i for i in x1])
  return list(new)

test_output = [(test(i)) for i in validation_dataset]



In [0]:
print("-------new_output--------")
print("violations: %d"  % violations(test_output))
print(span_stats(test_output))
print(" ")
print("-----baseline_output-----")
print("violations: %d"  % violations(baseline_output))
print(span_stats(baseline_output))

-------new_output--------
violations: 0
{'precision': 0.26, 'recall': 0.3023255813953488, 'f1': 0.27956989247311825}
 
-----baseline_output-----
violations: 30
{'precision': 0.2727272727272727, 'recall': 0.20930232558139536, 'f1': 0.23684210526315788}
